In [1]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

In [2]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(ann_file, lab_file):
    in_file = open(ann_file, 'r')
    out_file = open(lab_file, 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text) 
    classes = [x.find('name').text for x in root.iter('object')]
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
    return classes

In [ ]:
path = '/home/neo/Desktop/DATA/for_ocr'
im_dirs = ['train', 'val']
an_dir = ['annot_train', 'annot_val']


for i in range(len(im_dirs)):
    full_dir_path = path+'/'+im_dirs[i]
    output_img_path = full_dir_path +'/'+'images/'
    output_lab_path = full_dir_path +'/'+'labels/'
    
    try:
        if not os.path.exists(output_img_path) or os.path.exists(output_lab_path):
            os.makedirs(output_img_path)
            os.makedirs(output_lab_path)
    except:
        pass
        
    image_paths = [full_dir_path+'/'+x.split('.')[0] for x in os.listdir(full_dir_path)]
    label_path = [output_lab_path+x.split('.')[0] for x in os.listdir(full_dir_path)]
    annot_files = [path+'/'+an_dir[i]+'/'+x.split('.')[0] for x in os.listdir(path+'/'+an_dir[i])]
    for ann in annot_files:
        annotation_file = ann+'.xml'
        label_file = output_lab_path+ann.split('/')[-1]+'.txt'
        classes = convert_annotation(annotation_file, label_file)

with open ('data.yaml', 'w') as yml:
    yml.writelines([f"train: {'../train'}",'\n', f"val: {'../val'}", '\n',f'nc: {len(classes)}', '\n',f"names: {classes}"])

In [3]:
path = '/home/neo/Desktop/DATA/adar_data/data'

img_path = path +'/'+'images/'
output_lab_path = path +'/'+'labels/'
an_dir = 'annotations'

try:
    if not os.path.exists(output_lab_path):
        os.makedirs(output_lab_path)
except:
    pass
    
image_paths = [img_path+'/'+x.split('.')[0] for x in os.listdir(img_path)]
label_path = [output_lab_path+x.split('.')[0] for x in os.listdir(img_path)]
annot_files = [path+'/'+an_dir+'/'+x.split('.')[0] for x in os.listdir(path+'/'+an_dir)]
for ann in annot_files:
    annotation_file = ann+'.xml'
    label_file = output_lab_path+ann.split('/')[-1]+'.txt'
    classes = convert_annotation(annotation_file, label_file)

with open ('data.yaml', 'w') as yml:
    yml.writelines([f"train: {'../train'}",'\n', f"val: {'../val'}", '\n',f'nc: {len(classes)}', '\n',f"names: {classes}"])

In [ ]:
import os
path = '/home/neo/Desktop/DATA/adar_data/data'
output_lab_path = path +'/'+'labels/'
an_dir = 'annotations'

img_path = path +'/'+'images/'
image_paths = [img_path+x.split('.')[0] for x in os.listdir(img_path)]

label_path = [output_lab_path+x.split('.')[0] for x in os.listdir(img_path)]

annot_files = [path+'/'+an_dir+'/'+x.split('.')[0] for x in os.listdir(path+'/'+an_dir)]
